In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.internals.construction import to_arrays
from prompt_toolkit.utils import to_int
from pulp import *

## Cost matrix

In [32]:
cost_matrix = pd.read_csv('cost_matrix.csv').to_numpy()

In [33]:
# Number of shops to visit
n = cost_matrix.shape[0]

# Calculate M
M = 0
for i in range(n):
    for j in range(n):
        M += cost_matrix[i,j]

# M cost for the path i->i
for i in range(n):
    cost_matrix[i,i] = M + 1

## Shop demands

In [34]:
shop_demands = pd.read_csv('shop_demands.csv').to_numpy()

## Vehicle routing problem

In [35]:
# Vehicle capacity
van_capacity = 300

#Set N of shops
N = [i for i in range(1,n)]

#Set of shops and storage
V = [0] + N

In [36]:
# Creates a list of tuples containing all the possible routes for transport
A=[(i,j) for i in V for j in V if i!=j]

In [37]:
# A set of variables x is created to contain the vehicle routes variables (binary var)
x = {}
for a in A:
    x[a] = LpVariable("x(%s,%s)" %a, cat=LpBinary)

# A set of variables u is created to contain the continuous delivered quantity
u = {}
for i in N:
    u[i] = LpVariable("u(%s)" %i, shop_demands[i], van_capacity)

In [38]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("CVRP_Problem",LpMinimize)

In [39]:
# The objective function is added to 'prob' first
prob += lpSum(cost_matrix[i,j]*x[i,j] for (i,j) in A)

In [40]:
#Constraint 1 : imposes that exactly one arc enters each customer node
one_visit_out = {}
for i in N:
    one_visit_out[i] = lpSum([x[i,j] for j in V if j!=i]) == 1.0
    prob += one_visit_out[i]

In [41]:
#Constraint 2 : imposes that exactly one arc leaves each customer node
one_visit_in = {}
for j in N:
    one_visit_in[j] = lpSum([x[i,j] for i in V if i!=j]) == 1.0
    prob += one_visit_in[j]

In [42]:
#Constraint 4 : imposes the amount of flow that leaves the depot should be identical
#with the flow returns to the depot   
depot_node = lpSum([x[0,j] for j in N]) == lpSum([x[i,0] for i in N])
prob += depot_node

In [43]:
#Constraint 5 : vehicle capacity constraint and sub tour elimination constraint
sub_tours_cap = {}
for i in N:
    for j in N:
        if i != j:
            sub_tours_cap[i,j] = u[i] - u[j] + van_capacity * x[i,j] <= van_capacity - shop_demands[j]
            prob += sub_tours_cap[i,j]

In [54]:
# The problem is solved using PuLP's choice of Solver (ici Cplex, retourne 1 si résolu, -1 sinon)
prob.solve(GUROBI(msg=False))

1

In [45]:
# The status of the solution is printed to the screen (optimal ou pas)
print("Status:", LpStatus[prob.status])

Status: Optimal


In [46]:
# Each of the variables is printed with it's resolved optimum value
for key,val in x.items():
    if val.varValue==1.0:
        print(val,"=",val.varValue)

x(0,2) = 1.0
x(0,4) = 1.0
x(1,0) = 1.0
x(2,3) = 1.0
x(3,0) = 1.0
x(4,1) = 1.0


In [48]:
# The optimised objective function value is printed to the screen
print("Total Cost = ", value(prob.objective))

Total Cost =  10.0


In [49]:
# Afficher les acrs correspondant à la solution
active_arcs = [key for key,val in x.items() if val.varValue >=0.9 ]
print(active_arcs)

[(0, 2), (0, 4), (1, 0), (2, 3), (3, 0), (4, 1)]


In [52]:
# Each of the variables is printed with it's resolved optimum value
for key,val in u.items():
    print(val,"=",val.varValue)

u(1) = 160.0
u(2) = 80.0
u(3) = 230.0
u(4) = 120.0
